# Analisis de Egresos Hospitalarios

In [6]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)

<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [7]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [8]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)


# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Días de Estada promedio (dias_estada_promedio)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [9]:


AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)

# Analisis de Estratos

In [74]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION


In [75]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)


ComputeError: LogicalPlan already failed with error: 'ColumnNotFound(ErrString("region_pais\n\nError originated just after this operation:\n WITH_COLUMNS:\n [col(\"INTERV_Q\").map(), col(\"CONDICION_EGRESO\").map(), col(\"GLOSA_REGION_RESIDENCIA\").map()]\n  FILTER col(\"DIAG1\").is_in([Series[DIAG1].alias(\"DIAG1\")]) FROM\n  RECHUNK\n    UNION:\n    PLAN 0:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2001.csv\n        PROJECT */37 COLUMNS\n    PLAN 1:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2002.csv\n        PROJECT */37 COLUMNS\n    PLAN 2:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2003.csv\n        PROJECT */37 COLUMNS\n    PLAN 3:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2004.csv\n        PROJECT */37 COLUMNS\n    PLAN 4:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2005.csv\n        PROJECT */37 COLUMNS\n    PLAN 5:\n\n        CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2006.csv\n        PROJECT */37 COLUMNS\n  

In [76]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())


# Analisis Sociodemográfico

In [69]:
df.select(
    pl.col('EDAD_CATEGORIA')
).fetch(10)

ComputeError: LogicalPlan already failed with error: 'ColumnNotFound(ErrString("region_pais\n\nError originated just after this operation:\n WITH_COLUMNS:\n [col(\"INTERV_Q\").map(), col(\"CONDICION_EGRESO\").map(), col(\"GLOSA_REGION_RESIDENCIA\").map()]\n   WITH_COLUMNS:\n   [Series[EDAD_CATEGORIA].alias(\"EDAD_CATEGORIA\")]\n    FILTER col(\"DIAG1\").is_in([Series[DIAG1].alias(\"DIAG1\")]) FROM\n    RECHUNK\n      UNION:\n      PLAN 0:\n\n          CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2001.csv\n          PROJECT */37 COLUMNS\n      PLAN 1:\n\n          CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2002.csv\n          PROJECT */37 COLUMNS\n      PLAN 2:\n\n          CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2003.csv\n          PROJECT */37 COLUMNS\n      PLAN 3:\n\n          CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2004.csv\n          PROJECT */37 COLUMNS\n      PLAN 4:\n\n          CSV SCAN input\\utf-8\\Egresos_Hospitalarios_2005.csv\n          PROJECT */37 COLUMNS\n 